# quick_pp

In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from quick_pp.objects import Project

# Load well from saved file
project_name = "MOCK_carbonate"
project_path = rf"data\04_project\{project_name}.qppp"
project = Project().load(project_path)
project.get_well_names()

# Batch Interpretation

In [ ]:
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import MinMaxScaler

from quick_pp.lithology.carbonate import Carbonate
from quick_pp.porosity import neu_den_xplot_poro, density_porosity, rho_matrix, normalize_volumetric
from quick_pp.qaqc import badhole_flagging, mask_outside_threshold, neu_den_xplot_hc_correction
from quick_pp.saturation import estimate_temperature_gradient, estimate_rw_temperature_salinity, archie_saturation
from quick_pp.ressum import calc_reservoir_summary
from quick_pp.rock_type import estimate_vsh_gr, calc_r35_perm, rock_typing, calc_fzi_perm
from quick_pp.core_calibration import sw_shf_leverett_j, sw_skelt_harrison

# Load FZI ROCK_FLAG
with open(r'data\04_project\MOCK_carbonate\outputs\fzi_rt_model.qppm', 'rb') as file:
    fzi_rt_model = pickle.load(file)

# Load R35 ROCK_FLAG
with open(r'data\04_project\MOCK_carbonate\outputs\r35_rt_model.qppm', 'rb') as file:
    r35_rt_model = pickle.load(file)

# Load R35 model
with open(r'data\04_project\MOCK_carbonate\outputs\r35_model.qppm', 'rb') as file:
    r35_model = pickle.load(file)

# Load FZI model
with open(r'data\04_project\MOCK_carbonate\outputs\fzi_model.qppm', 'rb') as file:
    fzi_model = pickle.load(file)

In [ ]:
WATER_CONTACT = {
    'HW-3': 8220,
    'HW-4': 8220,
    'HW-5': 8243,
    'HW-6': 8082,
    'HW-7': 8533,
    'HW-8': 8658,
    'HW-9': 8658,
    'HW-10': 8554,
    'HW-24': 8552,
    'HW-25': 8421,
    'HW-26': 8573,
    'HW-27': 8656,
    'HW-28': 8564,
    'HW-29': 8377,
    'HW-30': 8543,
    'HW-31': 8594,
    'HW-32': 8543,
}

In [ ]:
carbonate_type =  'limestone'  # 'dolostone'  #
model = 'single'  # 'double'  #
method = 'neu_den'  # 'pef_den'  #

df = project.get_data()  # [['WELL_NAME', 'DEPTH', 'ZONES', 'GR', 'RT', 'NPHI', 'RHOB', 'CALI', 'BS']]
# df = all_data.copy()
for well_name, well_data in df.groupby('WELL_NAME'):
    # Load well object
    well = project.get_well(well_name)
    well_data['CPORE'] = well_data['CORE_POR'] / 100
    well_data['CPERM'] = well_data['CORE_PERM']

    # Clean up data
    well_data = badhole_flagging(well_data)
    # well_data = mask_outside_threshold(well_data, fill=True)

    # Initialize lithology model
    args = {
        'litho_model': 'carb',
        'dry_calc_point': (.0, 2.71),
        'dry_clay_point': (.3, 2.7),
        'sw_water_salinity': 2e5,
        'sw_m': 2.2,
        'sw_n': 2,
        'hc_corr_angle': 30,
        'hc_buffer': 0.01,
        'ressum_cutoffs': dict(
            VSHALE=.5,
            PHIT=0,
            SWT=1
        ),
    }
    carb_model = Carbonate(**args)
    vclw_un, vcalc_un, vdolo_un = carb_model.estimate_lithology(
        well_data['NPHI'], well_data['RHOB'],
        model=model, method=method, carbonate_type=carbonate_type
    )
    args.update(carb_model.__dict__)
    well.update_config(args)  # Save lithology model to well

    # Set skip HC correction or not
    skip_hc_correction = False
    if skip_hc_correction:
        nphihc, rhobhc = well_data['NPHI'], well_data['RHOB']
    else:
        # Implement hydrocarbon correction
        nphihc, rhobhc, hc_flag = neu_den_xplot_hc_correction(
            well_data['NPHI'], well_data['RHOB'],
            dry_min1_point=args['dry_calc_point'],
            dry_clay_point=args['dry_clay_point'],
            corr_angle=args['hc_corr_angle'], buffer=args['hc_buffer']
        )

        # Estimate lithology
        carb_model = Carbonate(**args)
        vclw_un, vcalc_un, vdolo_un = carb_model.estimate_lithology(
            nphihc, rhobhc, model=model, method=method, carbonate_type=carbonate_type
        )

    # Estimate porosity
    phit = neu_den_xplot_poro(
        nphihc, rhobhc, model='carb',
        dry_min1_point=args['dry_calc_point'],
        dry_clay_point=args['dry_clay_point'],
    )

    # PHID needs unnormalized lithology
    rho_ma = rho_matrix(vclay=vclw_un, vcalc=vcalc_un, vdolo=vdolo_un)
    phid = density_porosity(rhobhc, rho_ma)
    
    # Fill missing values in phit with phid
    phit = np.where(np.isnan(phit), phid, phit)
    
    # Normalize lithology
    volumes = dict(
        vclw=vclw_un, vcalc=vcalc_un, vdolo=vdolo_un
    )
    volumes = normalize_volumetric(phit, **volumes)
    vclw, vcalc, vdolo = volumes['vclw'], volumes['vcalc'], volumes['vdolo']
    
    well_data['LOG_RT'] = np.log10(well_data['RT'])
    well_data['NDI'] = (2.95 - well_data['RHOB']) / 1.95
    well_data['GRN'] = MinMaxScaler().fit_transform(well_data[['GR']])
    input_features = ['NPHI', 'NDI', 'LOG_RT', 'GRN']
    
    use_r35 = False
    use_j = False
    if use_r35:
        # Predict permeability
        r35_ml = 10**(r35_model.predict(well_data[input_features]))
        perm_ml = calc_r35_perm(r35_ml, phit)

        # Predict rock type
        rock_flag_ml = r35_rt_model.predict(well_data[input_features])
        if use_j:
            shf_params = {
                1.0: {'Sample': 86.0, 'a': 0.001, 'b': 2.288},
                2.0: {'Sample': 75.0, 'a': 0.002, 'b': 2.017},
                3.0: {'Sample': 63.0, 'a': 0.007, 'b': 2.034},
                4.0: {'Sample': 22.0, 'a': 0.013, 'b': 1.44},
                5.0: {'Sample': 11.0, 'a': 0.004, 'b': 2.33},
                6.0: {'Sample': 8.0, 'a': 0.003, 'b': 2.042},
                7.0: {'Sample': 9.0, 'a': 0.032, 'b': 0.578},
                8.0: {'Sample': 6.0, 'a': 0.016, 'b': 1.141},
                9.0: {'Sample': 3.0, 'a': 0.016, 'b': 1.221}
            }
        else:
            shf_params = {
                1.0: {'Sample': 85.0, 'a': 1.002, 'b': 0.086, 'c': 0.916, 'd': -0.81},
                2.0: {'Sample': 79.0, 'a': 1.92, 'b': 0.024, 'c': 0.051, 'd': -1.571},
                3.0: {'Sample': 44.0, 'a': 0.971, 'b': 0.566, 'c': 1.008, 'd': -2.226},
                4.0: {'Sample': 27.0, 'a': 0.966, 'b': 1.321, 'c': 1.065, 'd': -3.767},
                5.0: {'Sample': 10.0, 'a': 0.992, 'b': 3.302, 'c': 0.897, 'd': -5.02},
                6.0: {'Sample': 7.0, 'a': 0.961, 'b': 2.575, 'c': 0.917, 'd': -7.721},
                7.0: {'Sample': 9.0, 'a': 0.992, 'b': 12.142, 'c': 1.961, 'd': -6.231},
                8.0: {'Sample': 6.0, 'a': 0.985, 'b': 13.172, 'c': 0.979, 'd': -4.206},
                9.0: {'Sample': 5.0, 'a': 0.956, 'b': 16.482, 'c': 1.765, 'd': -1.804}
            }
    else:
        # Predict permeability
        fzi_ml = 10**(fzi_model.predict(well_data[input_features]))
        perm_ml = calc_fzi_perm(fzi_ml, phit)

        # Predict rock type
        rock_flag_ml = fzi_rt_model.predict(well_data[input_features])
        if use_j:
            shf_params = {   
                1.0: {'Sample': 68.0, 'a': 0.007, 'b': 1.426},
                2.0: {'Sample': 73.0, 'a': 0.005, 'b': 2.844},
                3.0: {'Sample': 63.0, 'a': 0.008, 'b': 1.914},
                4.0: {'Sample': 46.0, 'a': 0.001, 'b': 3.399},
                5.0: {'Sample': 22.0, 'a': 0.009, 'b': 1.509},
                6.0: {'Sample': 13.0, 'a': 0.015, 'b': 1.16},
                7.0: {'Sample': 11.0, 'a': 0.019, 'b': 1.021},
                8.0: {'Sample': 7.0, 'a': 0.018, 'b': 0.849},
                9.0: {'Sample': 5.0, 'a': 0.014, 'b': 0.771}
            }
        else:
            shf_params = {   
                1.0: {'Sample': 84.0, 'a': 0.995, 'b': 0.266, 'c': 1.397, 'd': -0.291},
                2.0: {'Sample': 85.0, 'a': 1.002, 'b': 0.088, 'c': 0.928, 'd': -0.809},
                3.0: {'Sample': 49.0, 'a': 0.991, 'b': 0.989, 'c': 1.48, 'd': -0.821},
                4.0: {'Sample': 44.0, 'a': 0.978, 'b': 0.57, 'c': 0.982, 'd': -2.227},
                5.0: {'Sample': 61.0, 'a': 0.987, 'b': 0.696, 'c': 1.155, 'd': -0.557},
                6.0: {'Sample': 27.0, 'a': 0.984, 'b': 0.979, 'c': 0.847, 'd': -4.094},
                7.0: {'Sample': 12.0, 'a': 0.994, 'b': 3.441, 'c': 1.238, 'd': -9.85},
                8.0: {'Sample': 7.0, 'a': 0.988, 'b': 2.027, 'c': 0.731, 'd': -8.247},
                9.0: {'Sample': 9.0, 'a': 0.997, 'b': 11.406, 'c': 1.836, 'd': -6.964}
            }

    # Estimate water saturation
    temp_grad = estimate_temperature_gradient(well_data['DEPTH'], 'metric')  # Arbitrarily use MD instead of TVD
    water_salinity = args['sw_water_salinity']
    rw = estimate_rw_temperature_salinity(temp_grad, water_salinity)
    m = args['sw_m']
    swt = archie_saturation(well_data['RT'], rw, phit, m=m)
    swt = swt.clip(0, 1)
    
    # Estimate SHF
    ift = 32
    theta = 30
    ghc = .837
    gw = 1.135

    fwl = WATER_CONTACT[well_name]

    if use_j:
        a = pd.Series(rock_flag_ml).map(shf_params).apply(lambda x: x['a'])
        b = pd.Series(rock_flag_ml).map(shf_params).apply(lambda x: x['b'])
        shf = sw_shf_leverett_j(
            perm_ml, phit, well_data['DEPTH'], gw=gw, ghc=ghc,
            fwl=fwl, ift=ift, theta=theta, a=a, b=b
        )
    else:
        a = pd.Series(rock_flag_ml).map(shf_params).apply(lambda x: x['a'])
        b = pd.Series(rock_flag_ml).map(shf_params).apply(lambda x: x['b'])
        c = pd.Series(rock_flag_ml).map(shf_params).apply(lambda x: x['c'])
        d = pd.Series(rock_flag_ml).map(shf_params).apply(lambda x: x['d'])
        shf = sw_skelt_harrison(well_data.DEPTH, fwl, a, b, c, d)

    # Estimate reservoir summary
    well_data['ZONES'] = 'ALL'
    ressum_df = calc_reservoir_summary(well_data.DEPTH, vclw, phit, swt, perm_ml,
                                       zones=well_data['ZONES'], cutoffs=args['ressum_cutoffs'])

    # Update data in the project
    well_data['NPHI_HC'] = nphihc
    well_data['RHOB_HC'] = rhobhc
    # well_data['HC_FLAG'] = hc_flag
    well_data['VCALC'] = vcalc
    well_data['VDOLO'] = vdolo
    well_data['VCLW'] = vclw
    well_data['PHID'] = phid
    well_data['PHIT'] = phit
    well_data['SWT'] = swt
    well_data['SHF'] = np.where(shf < 1.5, shf, 1.5)
    well_data['BVW'] = phit * swt
    well_data['PERM'] = perm_ml
    well_data['VHC'] = phit * (1 - swt)
    well_data['ROCK_FLAG'] = rock_flag_ml
    well_data['ROCK_FLAG'] = well_data['ROCK_FLAG'].astype(int)

    # Save the results to the well
    well.update_data(well_data)
    well.update_ressum(ressum_df)
    project.save_well(well)
    # break


In [ ]:
from quick_pp.plotter import plotly_log
import os

df = project.get_data()
folder = rf"outputs\{project_name}\interpreted_logs"
os.makedirs(folder, exist_ok=True)
for well, well_data in df.groupby("WELL_NAME"):
    # Plot the results
    fig = plotly_log(well_data, depth_uom='ft')
    # fig.show(config=dict(scrollZoom=True))
    fig.write_html(rf"{folder}\{well}_log.html", config=dict(scrollZoom=True))

# QC the results

In [ ]:
df = project.get_data()

In [ ]:
df.columns

In [ ]:
from quick_pp.qaqc import quick_compare
from quick_pp.utils import zone_flagging

df['VSHALE'] = df['VCLW']
# df = zone_flagging(df)
compare_df, fig = quick_compare(df, return_fig=True)
compare_df = compare_df[compare_df['FLAG'] == 'all']

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(nrows=len(df['WELL_NAME'].unique()), ncols=1, figsize=(20, len(df['WELL_NAME'].unique())), sharex=True)

for ax, (well, well_data) in zip(axes, df.groupby("WELL_NAME")):
    ax.plot(well_data['DEPTH'], well_data['SWT'], label='SWT')
    ax.plot(well_data['DEPTH'], well_data['SHF'], label='SHF')
    # ax.set_ytitle(well)
    ax.set_ylabel(well)
    ax.set_ylim(0, 1)
    ax.legend()

plt.xlabel('DEPTH')
plt.show()

In [ ]:
import pandas as pd
from sklearn.metrics import (r2_score, mean_absolute_percentage_error, mean_absolute_error,
                             mean_squared_error, root_mean_squared_error)

vars_dict = {
    'PHIT': 'CPORE',
    'PERM': 'CPERM',
    'SWT': 'SHF'
}

for var, compare_var in vars_dict.items():
    data = df[[var, compare_var]].dropna()
    r2 = r2_score(data[compare_var], data[var])
    mape = mean_absolute_percentage_error(data[compare_var], data[var])
    mae = mean_absolute_error(data[compare_var], data[var])
    rmse = root_mean_squared_error(data[compare_var], data[var])
    mse = mean_squared_error(data[compare_var], data[var])
    print(f"{var} R2: {r2:.2f}, MAPE: {mape:.2f}, MAE: {mae:.2f}, RMSE: {rmse:.2f}, MSE: {mse:.2f}")

results = []
for well_name, well_data in df.groupby('WELL_NAME'):
    well_results = {'WELL_NAME': well_name}
    for var, compare_var in vars_dict.items():
        data = well_data[[var, compare_var]].dropna()
        r2 = r2_score(data[compare_var], data[var])
        mape = mean_absolute_percentage_error(data[compare_var], data[var])
        mae = mean_absolute_error(data[compare_var], data[var])
        rmse = root_mean_squared_error(data[compare_var], data[var])
        mse = mean_squared_error(data[compare_var], data[var])
        well_results[f"{var}_R2"] = r2
        well_results[f"{var}_MAPE"] = mape
        well_results[f"{var}_MAE"] = mae
        well_results[f"{var}_RMSE"] = rmse
        well_results[f"{var}_MSE"] = mse
    results.append(well_results)

results_df = pd.DataFrame(results).sort_values('PHIT_MAE')
results_df.set_index('WELL_NAME').T

In [ ]:
import matplotlib.pyplot as plt

colors = plt.cm.tab20(np.linspace(0, 1, len(df['WELL_NAME'].unique())))

for i, (well_name, well_data) in enumerate(df.groupby('WELL_NAME')):
    plt.scatter(well_data['CPORE'], well_data['PHIT'], marker='.', label=well_name, color=colors[i])
plt.title('Core vs Estimated Porosity')
plt.xlabel('Core Porosity')
plt.ylabel('Estimated Porosity')
plt.xlim(0, 0.4)
plt.ylim(0, 0.4)
plt.legend(bbox_to_anchor=(1.04, 1), loc="upper left")

In [ ]:
import matplotlib.pyplot as plt

colors = plt.cm.tab20(np.linspace(0, 1, len(df['WELL_NAME'].unique())))

for i, (well_name, well_data) in enumerate(df.groupby('WELL_NAME')):
    plt.scatter(well_data['CPERM'], well_data['PERM'], marker='.', label=well_name, color=colors[i])
plt.title('Core vs Estimated Permeability')
plt.xlabel('Core Permeability')
plt.ylabel('Estimated Permeability')
plt.xscale('log')
plt.yscale('log')
plt.xlim(1e-3, 1e4)
plt.ylim(1e-3, 1e4)
plt.legend(bbox_to_anchor=(1.04, 1), loc="upper left")

In [ ]:
sorting_dict = { 
    'HW-10': 3,
    'HW-24': 12,
    'HW-25': 13,
    'HW-26': 15,
    'HW-27': 2,
    'HW-28': 5,
    'HW-29': 6,
    'HW-3': 11,
    'HW-30': 4,
    'HW-31': 8,
    'HW-32': 9,
    'HW-4': 16,
    'HW-5': 17,
    'HW-6': 14,
    'HW-7': 10,
    'HW-8': 7,
    'HW-9': 1
}
compare_df = compare_df[compare_df['FLAG'] == 'all'].copy()
compare_df['SORT'] = compare_df['WELL_NAME'].map(sorting_dict)
compare_df.sort_values('SORT', inplace=True)

In [ ]:
compare_df.plot(kind='bar', x='WELL_NAME', y='AV_SWT', figsize=(10, 3), legend=False, title='Average SWT')

In [ ]:
compare_df.plot(kind='bar', x='WELL_NAME', y='AV_PHIT', figsize=(10, 3), legend=False, title='Average PHIT')

In [ ]:
results_df['SORT'] = results_df['WELL_NAME'].map(sorting_dict)
results_df.sort_values('SORT', inplace=True)

metric = ['R2', 'MAE', 'RMSE', 'MSE']
for m in metric:
    results_df.plot(kind='bar', x='WELL_NAME', y=[f'PHIT_{m}'], figsize=(10, 3), title=f'PHIT {m} by well', legend=False)

In [ ]:
compare_df.plot(kind='bar', x='WELL_NAME', y='PERM_AM', figsize=(10, 3), legend=False, title='Arithmetic Mean PERM',
                logy=True, ylim=(1e-2, 1e2))
compare_df.plot(kind='bar', x='WELL_NAME', y='PERM_GM', figsize=(10, 3), legend=False, title='Geometric Mean PERM',
                logy=True, ylim=(1e-2, 1e2))
compare_df.plot(kind='bar', x='WELL_NAME', y='PERM_HM', figsize=(10, 3), legend=False, title='Harmonic Mean PERM',
                logy=True, ylim=(1e-2, 1e2))

In [ ]:
metric = ['R2', 'MAE', 'RMSE', 'MSE']
for m in metric:
    results_df.plot(kind='bar', x='WELL_NAME', y=[f'PERM_{m}'], figsize=(10, 3), title=f'PERM {m} by well', legend=False)